In [4]:
pip install paho-mqtt

     ---------------------------------------- 0.0/99.4 kB ? eta -:--:--
     ---------------------------------------- 99.4/99.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=65646 sha256=372ea6f391752de20e5bddd5f5f4029fbc3e232907d065af88f3b70a552fb691
  Stored in directory: c:\users\lee\appdata\local\pip\cache\wheels\29\ea\a5\ba9a63aaf4cd4e16e8a87ee31fb4d11b04ff5e1735d312619a
Successfully built paho-mqtt
Note: you may need to restart the kernel to use updated packages.


In [17]:
!pip install pymysql

  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 44.8/44.8 kB 2.3 MB/s eta 0:00:00


In [27]:
pip install --upgrade certifi

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# alarm_server.py

import paho.mqtt.client as mqtt
import pymysql
import json
from datetime import datetime, timedelta
import time
import threading

# MySQL 연결 설정
connection = pymysql.connect(
    host='project-db-campus.smhrd.com',
    port=3307,
    user='ttap',
    password='1234',
    database='ttap',
    cursorclass=pymysql.cursors.DictCursor
)

# 알람 목록을 저장할 리스트
alarms = []

# MQTT 브로커 설정
broker_address = "172.30.1.21"
broker_port = 1883

# MQTT 클라이언트 생성
client = mqtt.Client()

# 주기적으로 현재 시간을 확인하고 알람이 발생해야 하는지 여부를 판단하는 함수
def check_and_trigger_alarm_periodically():
    while True:
        get_alarms()
        current_time = datetime.now().replace(microsecond=0).time()  # 초 부분을 0으로 만듦

        for alarm in alarms:
            alarm_time = datetime.strptime(alarm['time'], "%H:%M").time()  # 알람의 초 부분을 무시
            if alarm_time == current_time and alarm['isOn']:
                # 라즈베리파이에게 알람 울리도록 메시지 전송
                print(f"Publishing triggered message to Dart/Mqtt_client/flutter/alarm")
                client.publish("Dart/Mqtt_client/flutter/alarm", "triggered")

        # 1초마다 확인
        time.sleep(5)

# MQTT 연결 시 호출되는 콜백 함수
def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    # 토픽 구독
    client.subscribe("Dart/Mqtt_client/flutter/alarm")
    client.subscribe("Dart/Mqtt_client/flutter/sql")
    client.subscribe("Dart/Mqtt_client/flutter/alarm/add")
    client.subscribe("Dart/Mqtt_client/flutter/alarm/update")
    client.subscribe("Dart/Mqtt_client/flutter/alarm/delete")
    client.subscribe("Dart/Mqtt_client/flutter/alarm/get")

# MQTT 메시지 수신 시 호출되는 콜백 함수
def on_message(client, userdata, msg):
    print(f"Received message: {msg.payload.decode()}")

    # 알람 추가 메시지 처리
    if msg.topic == "Dart/Mqtt_client/flutter/alarm/add":
        add_alarm(msg.payload.decode())
    # 알람 수정 메시지 처리
    elif msg.topic == "Dart/Mqtt_client/flutter/alarm/update":
        update_alarm(msg.payload.decode())
    # 알람 삭제 메시지 처리
    elif msg.topic == "Dart/Mqtt_client/flutter/alarm/delete":
        delete_alarm(msg.payload.decode())
    # 알람 목록 조회 메시지 처리
    elif msg.topic == "Dart/Mqtt_client/flutter/alarm/get":
        get_alarms()
    # SQL 문을 받아서 실행하고 결과를 MQTT로 전송
    elif msg.topic == "Dart/Mqtt_client/flutter/sql":
        execute_sql_query(msg.payload.decode())

# 알람 추가 함수
def add_alarm(payload):
    try:
        alarm_data = json.loads(payload)
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에 추가
            sql = "INSERT INTO alarms (time, isOn, isSelected) VALUES (%s, %s, %s)"
            cursor.execute(sql, (alarm_data['time'], alarm_data['isOn'], alarm_data['isSelected']))
            connection.commit()
        print(f"Alarm added: {alarm_data}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error adding alarm: {e}")

# 알람 수정 함수
def update_alarm(payload):
    try:
        alarm_data = json.loads(payload)
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에서 수정
            sql = "UPDATE alarms SET time=%s, isOn=%s, isSelected=%s WHERE id=%s"
            cursor.execute(sql, (alarm_data['time'], alarm_data['isOn'], alarm_data['isSelected'], alarm_data['id']))
            connection.commit()
        print(f"Alarm updated: {alarm_data}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error updating alarm: {e}")

# 알람 삭제 함수
def delete_alarm(payload):
    try:
        alarm_id = json.loads(payload)['id']
        with connection.cursor() as cursor:
            # 알람 정보를 데이터베이스에서 삭제
            sql = "DELETE FROM alarms WHERE id=%s"
            cursor.execute(sql, (alarm_id,))
            connection.commit()
        print(f"Alarm deleted: {alarm_id}")
    except (json.JSONDecodeError, pymysql.Error) as e:
        print(f"Error deleting alarm: {e}")

# 알람 목록 조회 함수
def get_alarms():
    global alarms
    try:
        with connection.cursor() as cursor:
            # 알람 목록을 데이터베이스에서 조회
            sql = "SELECT id, time, isOn, isSelected FROM alarms"
            cursor.execute(sql)
            alarms = cursor.fetchall()
        # 알람 목록을 MQTT로 전송
        client.publish("Dart/Mqtt_client/flutter/alarm/return", json.dumps(alarms))
    except pymysql.Error as e:
        print(f"Error getting alarms: {e}")

# SQL 문을 실행하고 결과를 MQTT로 전송하는 함수
def execute_sql_query(sql_query):
    try:
        with connection.cursor() as cursor:
            # SQL 실행
            cursor.execute(sql_query)
            # 결과 가져오기
            result = cursor.fetchall()
            # 결과를 MQTT 토픽에 전송
            client.publish("Dart/Mqtt_client/flutter/sql/return", json.dumps(result))
    except pymysql.Error as e:
        print(f"Error executing SQL: {e}")

# MQTT 클라이언트에 콜백 함수 등록
client.on_connect = on_connect
client.on_message = on_message

# MQTT 브로커에 연결
client.connect(broker_address, broker_port, 60)

# 알람을 주기적으로 확인하는 쓰레드 시작
alarm_thread = threading.Thread(target=check_and_trigger_alarm_periodically)
alarm_thread.start()

# 브로커 루프 시작
client.loop_forever()

Connected with result code 0
Received message: SELECT sleep_num FROM ttap.mibanddata
